<a href="https://colab.research.google.com/github/korman/DeepFaceLab_Colab/blob/master/DeepFaceLab_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

Nothing to say , just run run run  it

## 安装miniconda

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

!conda --version

!conda create -n deepfacelab python=3.9 -y
!conda activate deepfacelab

# Step 1 Upload your *workspace* to Google Drive


1. Create a directory called ***DeepFaceLab*** in the root directory of Google Cloud.
2. Upload ***workspace*** to ***DeepFaceLab***  
3. Mount Google Drive as folder
4. Enter the directory by command



In [1]:
#Mount Google Drive as folder

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Enter the directory by command
%cd /content/drive/My Drive/HaoLab/

/content/drive/My Drive/HaoLab


#Step 2  Install DFL


In [4]:
#Clone stable master branch(recommended)

!git clone https://github.com/korman/DeepFaceLab_Colab.git

#Pull updates from repository
!git pull


# Into DeepFaceLab_Colab
%cd /content/drive/My Drive/HaoLab/DeepFaceLab_Colab

#Install requirements

!pip install -r requirements.txt
!pip install --upgrade scikit-image

Cloning into 'DeepFaceLab_Colab'...
remote: Enumerating objects: 2771, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2771 (delta 1), reused 3 (delta 1), pack-reused 2765 (from 1)
Receiving objects: 100% (2771/2771), 248.91 MiB | 16.39 MiB/s, done.
Resolving deltas: 100% (1853/1853), done.
Updating files: 100% (195/195), done.
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/content/drive/My Drive/HaoLab/DeepFaceLab_Colab
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


# Step 3. Extract faces

In [5]:
# Into DeepFaceLab_Colab
%cd /content/drive/My Drive/HaoLab/DeepFaceLab_Colab

/content/drive/My Drive/HaoLab/DeepFaceLab_Colab


In [7]:
#Extract frames from DST video
!pip install ffmpeg-python
!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/

Traceback (most recent call last):
  File "/content/drive/MyDrive/HaoLab/DeepFaceLab_Colab/main.py", line 237, in <module>
    arguments.func(arguments)
  File "/content/drive/MyDrive/HaoLab/DeepFaceLab_Colab/main.py", line 165, in process_videoed_extract_video
    VideoEd.extract_video (arguments.input_file, arguments.output_dir, arguments.output_ext, arguments.fps)
  File "/content/drive/MyDrive/HaoLab/DeepFaceLab_Colab/mainscripts/VideoEd.py", line 13, in extract_video
    output_path.mkdir(exist_ok=True)
  File "/usr/lib/python3.11/pathlib.py", line 1116, in mkdir
    os.mkdir(self, mode)
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/HaoLab/workspace/data_dst'


In [ ]:
#Extract Face from data_dst

!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug

In [ ]:
#Extract frames from SRC video
!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/

In [ ]:
#Extract Face from data_src

!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug

# Step 4. Train model


*   Only SAE
*   Set model options on output field
*   You can't get preview in output. (for now)
*   Different samples are saved in SAE_history. You can see the last saved preview, if you run the block 'Show last preview ..', ut you have to stop training for this.
*   Also, you can see the preview if you download the image from SAE_history manually. It does not require to stop training.
*   I plan to make a preview for the notebook later
*   For stop training you  need manually interrupt execution. You get error in output, but model will be saved.


In [ ]:
# Running trainer.

!python main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE --no-preview

Running trainer.

Loading model...

Model first run. Enter model options as default for each run.
Write preview history? (y/n ?:help skip:n) : 
n
Target iteration (skip:unlimited/default) : 
0
Batch_size (?:help skip:0) : 
0
Feed faces to network sorted by yaw? (y/n ?:help skip:n) : 
n
Flip faces randomly? (y/n ?:help skip:y) : 
y
Src face scale modifier % ( -30...30, ?:help skip:0) : 
0
Resolution ( 64-256 ?:help skip:128) : 
128
Half or Full face? (h/f, ?:help skip:f) : 
f
Learn mask? (y/n, ?:help skip:y) : 
y
Optimizer mode? ( 1,2,3 ?:help skip:1) : 
1
AE architecture (df, liae, vg ?:help skip:df) : 
df
AutoEncoder dims (32-1024 ?:help skip:512) : 
512
Encoder/Decoder dims per channel (21-85 ?:help skip:42) : 
42
Use CA weights? (y/n, ?:help skip: n ) : 
n
Use lightweight encoder? (y/n, ?:help skip:n) : 
n
Use multiscale decoder? (y/n, ?:help skip:n) : 
n
Use pixel loss? (y/n, ?:help skip: n ) : 
n
Face style power ( 0.0 .. 100.0 ?:help skip:0.00) : 
0.0
Background style power ( 0.0

# Step 5. Convert frames

In [ ]:
#Convert
!python main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model SAE

In [ ]:
#Convert frames to result video

!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

# Others

You can see the results of each step on google Drive

https://drive.google.com/drive/my-drive